In [ ]:
# download most recent data from VV
# and merge into a single tidy CSV file

import os
import requests
import zipfile
import io
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

def create_base_output_folder(base_path="data"):
    timestamp = datetime.now().strftime("vv_capture_%Y-%m-%d_%H-%M-%S")
    folder_path = os.path.join(base_path, timestamp)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

def download_and_extract(year, base_folder):
    url = f"https://verifiedvoting.org/api/api_sandbox.php?advanced&year={year}&download=csv"
    year_folder = os.path.join(base_folder, str(year))
    os.makedirs(year_folder, exist_ok=True)

    try:
        print(f"{year}: Downloading...\n")
        response = requests.get(url, stream=True, timeout=60)
        response.raise_for_status()

        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            zip_file.extractall(year_folder)
        return f"{year}: Downloaded and extracted succesfully"
    except Exception as e:
        return f"{year}: Failed with error {e}"

base_folder = create_base_output_folder()

latest_year = 2026
even_years = list(range(2006, latest_year+1, 2))

with ThreadPoolExecutor(max_workers=4) as executor:  # adjust thread count as needed
    futures = [executor.submit(download_and_extract, year, base_folder) for year in even_years]

    for future in as_completed(futures):
        print(future.result() + '\n')

print('Downloads complete, merging data...\n')


def load_verifier_data(data_dir):
    """Load all verifier data files from the data directory."""
    all_data = []
    
    for folder in sorted(os.listdir(data_dir)):
        folder_path = os.path.join(data_dir, folder)
        machines = os.path.join(folder_path, "verifier-machines.csv")
        jurisdictions = os.path.join(folder_path, "verifier-jurisdictions.csv")
        
        if os.path.isfile(machines):
            df = pd.read_csv(machines, skiprows=1, index_col=False, converters={'FIPS code': str})  # Skip the first two descriptive rows
            df['Year'] = folder.split('_')[0]  # Extract the year from folder name

            jurisdictions = pd.read_csv(jurisdictions, skiprows=1, index_col=False, converters={'FIPS code': str})  # Skip the first two descriptive rows
            df = pd.merge(df, jurisdictions[['FIPS code', 'Registered Voters']],
                     on='FIPS code',
                     how='left')

            all_data.append(df)
    
    return pd.concat(all_data, ignore_index=True)


# Load data
df = load_verifier_data(base_folder)

# Convert columns to appropriate types
df['Year'] = df['Year'].astype(int)
df['First Year in Use'] = df['First Year in Use'].astype('Int64')

# Create a new column to identify new equipment

def is_new_equipment(row):
    return row['First Year in Use'] == row['Year']

df['New Equipment'] = df.apply(is_new_equipment, axis=1)
df['First Year in Use'] = abs(df['First Year in Use']) # some years are negative, make them positive
df['Years in Use'] = df['Year'] - df['First Year in Use']
df['Registered Voters'] = df['Registered Voters'].astype('Int64')

df.sort_values(by=["FIPS code", "Equipment Type", "Year", "Manufacturer", "Model"], inplace=True)

# convert yes/no to nullable bool
for col in df.columns:
    unique_vals = df[col].dropna().unique()
    if 'Yes' in unique_vals and 'No' in unique_vals:
        df[col] = df[col].map({'Yes': True, 'No': False})
        df[col] = df[col].astype('boolean')  # Convert to nullable boolean type

# Initialize the "year_retired" column to False
df["Final Year in Use"] = False

# Group by the combination of FIPS code, Manufacturer, and Model
grouped = df.groupby(["FIPS code", "Manufacturer", "Model"])

# Iterate over each group
for _, group in grouped:
    # Get the index of the last record in the sorted group
    last_idx = group.sort_values("Year").index[-1]
    # Mark as 'final_year_in_use' if this last record's Year is not latest_year
    if df.loc[last_idx, "Year"] != latest_year:
        df.loc[last_idx, "Final Year in Use"] = True

reorder = [('Year', 0), ('Registered Voters', 4), ('First Year in Use', 8), ('Years in Use', 9), ('New Equipment', 10), ('Final Year in Use', 11)]
for col, new_position in reorder:
    # Move the column to the new position
    col = df.pop(col)
    df.insert(new_position, col.name, col)

# Save the cleaned data to a new CSV file
output_file = os.path.join(base_folder, "cleaned_verifier_data.csv")
df.to_csv(output_file, index=False)

print(f"Data merged and saved to {output_file}\n")



Data merged and saved to data/vv_capture_2025-06-04_16-23-48/cleaned_verifier_data.csv

